In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import librosa as lb
import librosa.display as lbd
import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('/content/drive/MyDrive/hacker/git_practice/confusion_practice/train.csv')
val=pd.read_csv('/content/drive/MyDrive/hacker/git_practice/confusion_practice/val.csv')
train.head() #((5173, 3), (1725, 3))

,Unnamed: 0,pid,filename,disease
0,1572,132,132_2b2_Lr_mc_LittC2SE_1.wav,COPD
1,2266,143,143_1b1_Al_sc_Meditron_17.wav,Healthy
2,561,154,154_4b4_Pr_mc_AKGC417L_4.wav,COPD
3,3165,209,209_1b1_Tc_sc_Meditron_2.wav,Healthy
4,1349,130,130_2b2_Ll_mc_AKGC417L_7.wav,COPD


In [ ]:
ytrain=train.disease
yval=val.disease
yval

0       COPD
1       COPD
2       COPD
3       COPD
4       COPD
        ... 
1720    COPD
1721    COPD
1722    COPD
1723    COPD
1724    COPD
Name: disease, Length: 1725, dtype: object

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
ytrain=le.fit_transform(ytrain)
yval=le.transform(yval)

Feature Extract은 저장해둔 거 불러오기

In [ ]:
features_val = np.load('/content/drive/MyDrive/hacker/git_practice/confusion_practice/features_val.npz')

In [ ]:
features_train = np.load('/content/drive/MyDrive/hacker/git_practice/confusion_practice/features_train.npz')

In [ ]:
for i in features_val:
  print(i)

mfcc_val
cstft_val
mSpec_val


In [ ]:
for i in features_train:
  print(i)

mfcc_train
cstft_train
mSpec_train


In [ ]:
mfcc_val = features_val['mfcc_val']
cstft_val = features_val['cstft_val']
mSpec_val = features_val['mSpec_val']

In [ ]:
mfcc_train = features_train['mfcc_train']
cstft_train = features_train['cstft_train']
mSpec_train = features_train['mSpec_train']

In [ ]:
print(mfcc_train.shape)
print(cstft_train.shape)
print(mSpec_train.shape)

(5173, 20, 259)
(5173, 12, 259)
(5173, 128, 259)


모델

In [ ]:
mfcc_input=keras.layers.Input(shape=(20,259,1),name="mfccInput")
x=keras.layers.Conv2D(32,5,strides=(1,3),padding='same')(mfcc_input) # (20, 87, 32)
x=keras.layers.BatchNormalization()(x)
x=keras.layers.Activation(keras.activations.relu)(x)
x=keras.layers.MaxPooling2D(pool_size=2,padding='valid')(x) # (10, 43, 32)

x=keras.layers.Conv2D(64,3,strides=(1,2),padding='same')(x) # (10, 22, 64)
x=keras.layers.BatchNormalization()(x)
x=keras.layers.Activation(keras.activations.relu)(x)
x=keras.layers.MaxPooling2D(pool_size=2,padding='valid')(x) # (5, 11, 64)

x=keras.layers.Conv2D(96,2,padding='same')(x) # (5, 11, 96)
x=keras.layers.BatchNormalization()(x)
x=keras.layers.Activation(keras.activations.relu)(x)
x=keras.layers.MaxPooling2D(pool_size=2,padding='valid')(x) # (2, 5, 96)

x=keras.layers.Conv2D(128,2,padding='same')(x) # (2, 5, 128)
x=keras.layers.BatchNormalization()(x)
x=keras.layers.Activation(keras.activations.relu)(x)
mfcc_output=keras.layers.GlobalMaxPooling2D()(x) # (, 128)

mfcc_model=keras.Model(mfcc_input, mfcc_output, name="mfccModel")

In [ ]:
croma_input=keras.layers.Input(shape=(12,259,1),name="cromaInput")
x=keras.layers.Conv2D(32,5,strides=(1,3),padding='same')(croma_input)
x=keras.layers.BatchNormalization()(x)
x=keras.layers.Activation(keras.activations.relu)(x)
x=keras.layers.MaxPooling2D(pool_size=2,padding='valid')(x)


x=keras.layers.Conv2D(64,3,strides=(1,2),padding='same')(x)
x=keras.layers.Activation(keras.activations.relu)(x)
x=keras.layers.MaxPooling2D(pool_size=2,padding='valid')(x)

x=keras.layers.Conv2D(128,2,padding='same')(x)
x=keras.layers.BatchNormalization()(x)
x=keras.layers.Activation(keras.activations.relu)(x)
croma_output=keras.layers.GlobalMaxPooling2D()(x)

croma_model=keras.Model(croma_input, croma_output, name="cromaModel")

In [ ]:
mSpec_input=keras.layers.Input(shape=(128,259,1),name="mSpecInput")
x=keras.layers.Conv2D(32,5,strides=(2,3),padding='same')(mSpec_input) # (64, 87, 32)
x=keras.layers.BatchNormalization()(x)
x=keras.layers.Activation(keras.activations.relu)(x)
x=keras.layers.MaxPooling2D(pool_size=2,padding='valid')(x) # (32, 43, 32)

skip_x = x # (32, 43, 32)
skip_x = keras.layers.Conv2D(96,3,strides=(4,4),padding='same')(skip_x) #(8, 11, 96)

x=keras.layers.Conv2D(64,3,strides=(2,2),padding='same')(x) #(16, 22, 64)
x=keras.layers.BatchNormalization()(x)
x=keras.layers.Activation(keras.activations.relu)(x)
x=keras.layers.MaxPooling2D(pool_size=2,padding='valid')(x) #(8, 11, 64)

x=keras.layers.Conv2D(96,2,padding='same')(x) #(8, 11, 96)
x = keras.layers.Add()([x,skip_x])

x=keras.layers.BatchNormalization()(x)
x=keras.layers.Activation(keras.activations.relu)(x)
x=keras.layers.MaxPooling2D(pool_size=2,padding='valid')(x) #(4, 5, 96)

x=keras.layers.Conv2D(128,2,padding='same')(x) #(4, 5, 128)
x=keras.layers.BatchNormalization()(x)
x=keras.layers.Activation(keras.activations.relu)(x)
mSpec_output=keras.layers.GlobalMaxPooling2D()(x) #(, 128)

mSpec_model_1=keras.Model(mSpec_input, mSpec_output, name="mSpecModel_1")

In [ ]:
input_mfcc=keras.layers.Input(shape=(20,259,1),name="mfcc")
MFCC=mfcc_model(input_mfcc) #mfcc는 mfcc model에 대한 output

input_croma=keras.layers.Input(shape=(12,259,1),name="croma")
CROMA=croma_model(input_croma)

input_mSpec=keras.layers.Input(shape=(128,259,1),name="mspec")
MSPEC=mSpec_model_1(input_mSpec)


concat=keras.layers.concatenate([MFCC,CROMA,MSPEC])
hidden=keras.layers.Dropout(0.2)(concat)
hidden=keras.layers.Dense(50,activation='relu')(concat)
hidden=keras.layers.Dropout(0.3)(hidden)
hidden=keras.layers.Dense(25,activation='relu')(hidden)
hidden=keras.layers.Dropout(0.3)(hidden)
output=keras.layers.Dense(8,activation='softmax')(hidden)

net_1=keras.Model([input_mfcc,input_croma,input_mSpec], output, name="Net")

In [ ]:
net_1.summary()

Model: "Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 mfcc (InputLayer)              [(None, 20, 259, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 croma (InputLayer)             [(None, 12, 259, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 mspec (InputLayer)             [(None, 128, 259, 1  0           []                               
                                )]                                                              

In [ ]:
accuracy='sparse_categorical_accuracy'
sparseLoss=keras.losses.SparseCategoricalCrossentropy()

from keras import backend as K
K.clear_session()
net_1.compile(optimizer='nadam', loss=sparseLoss, metrics=[accuracy])
K.set_value(net_1.optimizer.learning_rate, 0.001)

In [ ]:
file_name = '/content/drive/MyDrive/hacker/git_practice/confusion_practice/net_1_1_checkpoint-001.h5'
my_callbacks_2 = [
    tf.keras.callbacks.EarlyStopping(patience=10),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=3, min_lr=1e-8,mode='min'), #미분하다 막히면 lr 변경해줌
    tf.keras.callbacks.ModelCheckpoint(file_name, monitor='val_loss', verbose=0, save_best_only=True,)
]

모델 저장한 거 불러오기

In [ ]:
model=keras.models.load_model(file_name)

In [ ]:
model.compile(optimizer='nadam', loss=sparseLoss,metrics=[accuracy])

In [ ]:
loss, acc = model.evaluate({"mfcc":mfcc_val,"croma":cstft_val,"mspec":mSpec_val},yval, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

54/54 - 11s - loss: 0.1334 - sparse_categorical_accuracy: 0.9635 - 11s/epoch - 198ms/step
Restored model, accuracy: 96.35%


classification_report 혹은 confusion_matrix를 어떻게 만들 수 있을까요??

In [ ]:
#rom sklearn.metrics import confusion_matrix, plot_confusion_matrix
#import matplotlib.pyplot as pl

In [ ]:
from sklearn.metrics import classification_report

y_pred = net_1.predict({"mfcc":mfcc_val,"croma":cstft_val,"mspec":mSpec_val})

target_names=['URTI', 'Healthy', 'Asthma', 'COPD', 'LRTI', 'Bronchiectasis', 'Pheumonia', 'Bronchiolitis']
clf_report = classification_report(yval, np.argmax(y_pred, axis = 1), target_names=target_names)
print(clf_report)

                precision    recall  f1-score   support

          URTI       0.00      1.00      0.00         1
       Healthy       0.00      0.00      0.00        26
        Asthma       0.00      0.00      0.00        40
          COPD       0.77      0.02      0.03      1437
          LRTI       0.00      0.00      0.00        81
Bronchiectasis       0.01      0.88      0.03         8
     Pheumonia       0.00      0.00      0.00        71
 Bronchiolitis       0.02      0.18      0.04        61

      accuracy                           0.02      1725
     macro avg       0.10      0.26      0.01      1725
  weighted avg       0.65      0.02      0.03      1725



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
